In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dialogue-act/scripts/mapping.pair
/kaggle/input/dialogue-act/data/utterance_dialogue_act.py
/kaggle/input/dialogue-act/data/raw_data/README.md
/kaggle/input/dialogue-act/data/raw_data/schema.json
/kaggle/input/dialogue-act/data/raw_data/dialog_acts.json
/kaggle/input/dialogue-act/data/raw_data/val/dialogues_002.json
/kaggle/input/dialogue-act/data/raw_data/val/dialogues_001.json
/kaggle/input/dialogue-act/data/raw_data/test/dialogues_002.json
/kaggle/input/dialogue-act/data/raw_data/test/dialogues_001.json
/kaggle/input/dialogue-act/data/raw_data/train/dialogues_011.json
/kaggle/input/dialogue-act/data/raw_data/train/dialogues_002.json
/kaggle/input/dialogue-act/data/raw_data/train/dialogues_007.json
/kaggle/input/dialogue-act/data/raw_data/train/dialogues_005.json
/kaggle/input/dialogue-act/data/raw_data/train/dialogues_008.json
/kaggle/input/dialogue-act/data/raw_data/train/dialogues_017.json
/kaggle/input/dialogue-act/data/raw_data/train/dialogues_004.json
/kaggle/inpu

In [2]:
import sys
sys.path.append('/kaggle/input/dialogue-act/scripts')
sys.path.append('/kaggle/input/dialogue-act/data')

<h1>Khai báo thư viện</h1>

In [3]:
import json
from transformers import BertTokenizer, BertModel
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import f1_score, classification_report

<h1>Hàm để chuẩn hóa các utterance</h1>

In [4]:
import re
import string

# Đọc các cặp từ viết tắt và từ thay thế từ file 'mapping.pair'
with open('/kaggle/input/dialogue-act/scripts/mapping.pair') as fin:
    replacements = []
    for line in fin.readlines():
        tok_from, tok_to = line.replace('\n', '').split('\t')
        replacements.append((' ' + tok_from + ' ', ' ' + tok_to + ' '))  # Đảm bảo từ được bao quanh bởi khoảng trắng

# Hàm thay thế từ viết tắt trong văn bản
def replace_abbreviations(text):
    text = ' ' + text + ' '  # Thêm khoảng trắng đầu và cuối để dễ xử lý
    for fromx, tox in replacements:
        text = text.replace(fromx, tox)  # Thay thế tất cả các từ viết tắt
    return text[1:-1]  # Loại bỏ khoảng trắng thừa ở đầu và cuối

# Hàm làm sạch văn bản (normalize)
def normalize(text):
    # Các bước chuẩn hóa văn bản của bạn (đã được rút gọn và tối giản)
    text = text.lower()  # Chuyển tất cả chữ thành chữ thường
    text = re.sub(r'^\s*|\s*$', '', text)  # Xóa khoảng trắng ở đầu và cuối

    # Các thay thế cố định khác
    text = re.sub(r"b&b", "bed and breakfast", text)
    text = re.sub(r"b and b", "bed and breakfast", text)

    # Xử lý số điện thoại và mã bưu điện (theo mẫu có sẵn)
    text = re.sub(r'(\d{3})[-.\s]?(\d{3})[-.\s]?(\d{4,5})', r'\1\2\3', text)  # Lưu số điện thoại không dấu

    # Xử lý thời gian (giả sử chúng ta muốn chuẩn hóa các giá trị thời gian)
    text = re.sub(r"\d{1,2}[:]\d{2}", "[value_time]", text)

    # Xử lý giá tiền (giả sử các giá trị tiền có định dạng như 100.00 hoặc 100,000.00)
    text = re.sub(r"\d{1,3}(?:,\d{3})*(?:\.\d{2})?", "[value_price]", text)

    # Các thay thế khác
    text = re.sub(r'[^\w\s]', '', text)  # Xóa dấu câu không cần thiết
    text = re.sub(r'\s+', ' ', text)  # Loại bỏ khoảng trắng dư thừa

    # Thực hiện thay thế từ viết tắt
    text = replace_abbreviations(text)

    return text

# Hàm token hóa câu
def tokenize_text(text):
    # Tách câu thành các từ bằng cách sử dụng phương pháp token hóa đơn giản (cách này có thể thay thế bằng một thư viện token hóa mạnh mẽ hơn như nltk hoặc spaCy nếu cần)
    tokens = text.split()
    return tokens

# Hàm chuẩn hóa văn bản và lấy các token từ câu
def process_utterance(text):
    normalized_text = normalize(text)  # Chuẩn hóa văn bản
    tokens = tokenize_text(normalized_text)  # Token hóa văn bản đã chuẩn hóa
    return ' '.join(tokens)

# Hàm chuẩn hóa văn bản và lấy các token từ câu
def process_text(text):
    normalized_text = normalize(text)  # Chuẩn hóa văn bản
    tokens = tokenize_text(normalized_text)  # Token hóa văn bản đã chuẩn hóa
    return tokens

# Hàm chính để xử lý từng utterance và trả về dictionary
def process_utterances(data):

    processed_data = {}
    for item in data:
        utterance = item['utterance']
        
        # Làm sạch và thay thế từ viết tắt trong utterance
        tokens = process_text(utterance)

        # Lưu trữ thông tin sau khi xử lý, sử dụng các token của utterance
        processed_data[' '.join(tokens)] = item['dialogue_acts']
    return processed_data

# Hàm kiểm tra có tồn tại ký tự đặc biệt trong văn bản không
def contains_special_chars(text):
    special_chars = string.punctuation
    return any(char in special_chars for char in text)

# Hàm làm sạch các ký tự đặc biệt
def clean_special_chars(text):
    return ''.join(char for char in text if char not in string.punctuation)

# Hàm chính để kiểm tra các đặc điểm của văn bản
def analyze_text(text):
    # Kiểm tra xem có chứa ký tự đặc biệt hay không
    if contains_special_chars(text):
        print("Text contains special characters.")
    else:
        print("Text does not contain special characters.")

    # Làm sạch văn bản
    cleaned_text = clean_special_chars(text)
    print(f"Cleaned text: {cleaned_text}")


if __name__ == "__main__":

    # Ví dụ dữ liệu
    data = [
        {
            "utterance": "I'm looking for a B&B near the park, do you have any available?",
            "dialogue_acts": {
                "Hotel-Request": [
                    ["location", "park"],
                    ["availability", "yes"]
                ]
            }
        },
        {
            "utterance": "Can you tell me the price for a room for two?",
            "dialogue_acts": {
                "Hotel-Request": [
                    ["time", "two"]
                ]
            }
        }
    ]

    # Xử lý dữ liệu
    processed_utterances = process_utterances(data)

    # In ra kết quả
    print("Processed Utterances:")
    for utterance, dialogue_acts in processed_utterances.items():
        print(f"Utterance: {utterance}")
        print(f"Dialogue Acts: {dialogue_acts}")

    # Kiểm tra các đặc điểm của một câu
    example_text = "I'm looking for a B&B near the park!"
    analyze_text(example_text)


Processed Utterances:
Utterance: im looking for a bed and breakfast near the park do you have any available
Dialogue Acts: {'Hotel-Request': [['location', 'park'], ['availability', 'yes']]}
Utterance: can you tell me the price for a room for 2
Dialogue Acts: {'Hotel-Request': [['time', 'two']]}
Text contains special characters.
Cleaned text: Im looking for a BB near the park


<h1>Hàm để load data gồm: utterance và dialogue act</h1>

In [5]:
# Function to load data from JSON files
def load_data_from_json(file_path):
    """
    Loads the dataset from a JSON file where each utterance is associated with dialogue acts and their attributes.
    
    Args:
        file_path (str): Path to the JSON file.
    
    Returns:
        List of tuples: Each tuple contains an utterance and its corresponding dialogue acts.
    """
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Extract utterances and dialogue acts
    utterance_dialogue_act_dict = {}
    for item in data:
        utterance = process_utterance(item['utterance'])
        dialogue_acts = item['dialogue_acts']
        # Convert dialogue acts into a list of labels (as an example of how you can format them)
        labels = []
        for act, attributes in dialogue_acts.items():
            for attribute in attributes:
                # Create a unique label for each dialogue act and its attributes (e.g., Train-Inform-departure)
                labels.append(f"{act}-{attribute[0]}")
        utterance_dialogue_act_dict[utterance] = labels

    return utterance_dialogue_act_dict
    

<h1>Khởi tạo các class:</h1>
<h3>+ DialogueActDataset</h3>
<h3>+ BaseClassifier</h3>
<h3>+ BERTClassifier</h3>

In [6]:
# Dataset class for token classification
class DialogueActDataset(Dataset):
    """
    PyTorch Dataset for Dialogue Act data.
    """
    def __init__(self, data, tokenizer, max_len, label_list):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.label_list = label_list
        self.num_classes = len(label_list)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        utterance, labels = self.data[idx]
        tokens = self.tokenizer(utterance, truncation=True, padding='max_length', max_length=self.max_len, return_tensors='pt')
        input_ids = tokens['input_ids'].squeeze(0)
        attention_mask = tokens['attention_mask'].squeeze(0)
        
        # Create a binary vector for labels with 1's at positions of the relevant acts
        label_ids = torch.zeros(self.num_classes)
        for label in labels:
            if label in self.label_list:
                label_index = self.label_list.index(label)
                label_ids[label_index] = 1  # Set 1 for the relevant label
        
        return input_ids, attention_mask, label_ids


In [7]:
# Base model class for sequence labelling
class BaseClassifier(nn.Module):
    """
    Base class for sequence classification models.
    """
    def __init__(self, input_dim, hidden_dim, num_labels):
        super(BaseClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_labels = num_labels
        self.classifier = nn.Linear(hidden_dim, num_labels)

    def forward(self, x):
        logits = self.classifier(x)
        return logits

In [8]:
# BERT-based classifier
class BERTClassifier(BaseClassifier):
    def __init__(self, encoder_name='bert-base-uncased', num_labels=10):
        bert_model = BertModel.from_pretrained(encoder_name)
        super().__init__(input_dim=bert_model.config.hidden_size, hidden_dim=bert_model.config.hidden_size, num_labels=num_labels)
        self.bert = bert_model

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return super().forward(outputs.last_hidden_state[:, 0, :])  # Use the first token (CLS token)

<h1>Huấn luyện mô hình và đánh giá mô hình</h1>

In [9]:
# Training and evaluation function
def train_and_evaluate_model(model, train_loader, val_loader, test_loader, optimizer, criterion, num_epochs=20, device='cpu'):
    model = model.to(device)
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for input_ids, attention_mask, labels in train_loader:
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            optimizer.zero_grad()
            logits = model(input_ids, attention_mask)  # Chuyển cả input_ids và attention_mask vào mô hình
            loss = criterion(logits, labels)  # BCEWithLogitsLoss cho multi-label classification
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {total_loss / len(train_loader):.4f}")

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for input_ids, attention_mask, labels in val_loader:
                input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
                logits = model(input_ids, attention_mask)  # Chuyển cả input_ids và attention_mask vào mô hình
                loss = criterion(logits, labels)
                val_loss += loss.item()

        print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {val_loss / len(val_loader):.4f}")

    # Evaluation trên test data
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for input_ids, attention_mask, labels in test_loader:
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            preds = torch.sigmoid(model(input_ids, attention_mask))  # Chuyển cả input_ids và attention_mask vào mô hình
            all_preds.append(preds)
            all_labels.append(labels)

    # Flatten predictions và labels
    all_preds = torch.cat(all_preds, dim=0)
    all_labels = torch.cat(all_labels, dim=0)

    # Threshold predictions (ví dụ: nếu sigmoid output > 0.5, coi là 1)
    all_preds = (all_preds > 0.5).float()

    # Tính toán F1-score cho từng lớp
    f1 = f1_score(all_labels.cpu(), all_preds.cpu(), average='macro')  # Tính F1-score với average là 'macro'
    print(f"F1 Score (Macro): {f1:.4f}")

    # Cũng có thể sử dụng classification_report để hiển thị các chỉ số như precision, recall và F1 cho mỗi lớp
    report = classification_report(all_labels.cpu(), all_preds.cpu(), target_names=[f"Class {i}" for i in range(all_labels.size(1))])
    print("Classification Report:")
    print(report)

    # Evaluation metric (ví dụ: accuracy)
    accuracy = (all_preds == all_labels).float().mean()
    print(f"Accuracy on test set: {accuracy:.4f}")

In [10]:
# Predict function to test with any utterance
def predict(model, utterance, tokenizer, label_list, max_len=50, device='cpu'):
    model.eval()  # Chuyển sang chế độ đánh giá
    tokens = tokenizer(utterance, truncation=True, padding='max_length', max_length=max_len, return_tensors='pt')
    input_ids = tokens['input_ids'].squeeze(0).to(device)
    attention_mask = tokens['attention_mask'].squeeze(0).to(device)

    with torch.no_grad():
        logits = model(input_ids.unsqueeze(0), attention_mask.unsqueeze(0))

    preds = torch.sigmoid(logits).cpu().numpy()
    preds = (preds > 0.5).astype(int)  # Chuyển kết quả thành 0 hoặc 1

    predicted_labels = [label_list[i] for i in range(len(preds[0])) if preds[0][i] == 1]
    return predicted_labels


In [11]:
# Main function
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load the train, validation, and test data from JSON files
    train_data = load_data_from_json('/kaggle/input/dialogue-act/data/processed_data/train.json')
    val_data = load_data_from_json('/kaggle/input/dialogue-act/data/processed_data/val.json')
    test_data = load_data_from_json('/kaggle/input/dialogue-act/data/processed_data/test.json')

    label_list = list(set([label for labels in train_data.values() for label in labels]))

    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    max_len = 50

    train_dataset = DialogueActDataset(list(train_data.items()), tokenizer, max_len, label_list)
    val_dataset = DialogueActDataset(list(val_data.items()), tokenizer, max_len, label_list)
    test_dataset = DialogueActDataset(list(test_data.items()), tokenizer, max_len, label_list)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16)
    test_loader = DataLoader(test_dataset, batch_size=16)

    bert_model = BERTClassifier(num_labels=len(label_list)).to(device)
    optimizer = optim.Adam(bert_model.parameters(), lr=1e-5)
    criterion = nn.BCEWithLogitsLoss()

    print("Training BERT Model")
    train_and_evaluate_model(bert_model, train_loader, val_loader, test_loader, optimizer, criterion, num_epochs=10, device=device)

    # Save the model
    torch.save(bert_model.state_dict(), '/kaggle/working/bert_model.pth')

    # Test with a new utterance
    utterance = "What type of food would you like?"
    predicted_labels = predict(bert_model, utterance, tokenizer, label_list, max_len, device)
    print(f"Predicted Dialogue Acts for the utterance: {predicted_labels}")

In [12]:
main()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

Training BERT Model
Epoch 1/10, Training Loss: 0.0521
Epoch 1/10, Validation Loss: 0.0138
Epoch 2/10, Training Loss: 0.0113
Epoch 2/10, Validation Loss: 0.0082
Epoch 3/10, Training Loss: 0.0081
Epoch 3/10, Validation Loss: 0.0071
Epoch 4/10, Training Loss: 0.0068
Epoch 4/10, Validation Loss: 0.0065
Epoch 5/10, Training Loss: 0.0061
Epoch 5/10, Validation Loss: 0.0063
Epoch 6/10, Training Loss: 0.0055
Epoch 6/10, Validation Loss: 0.0063
Epoch 7/10, Training Loss: 0.0050
Epoch 7/10, Validation Loss: 0.0062
Epoch 8/10, Training Loss: 0.0046
Epoch 8/10, Validation Loss: 0.0064
Epoch 9/10, Training Loss: 0.0042
Epoch 9/10, Validation Loss: 0.0066
Epoch 10/10, Training Loss: 0.0038
Epoch 10/10, Validation Loss: 0.0067


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


F1 Score (Macro): 0.5040


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.90      0.56      0.69        16
     Class 1       0.90      0.81      0.85        47
     Class 2       0.94      0.94      0.94       622
     Class 3       1.00      0.11      0.20        18
     Class 4       0.87      0.91      0.89       315
     Class 5       0.81      0.83      0.82       150
     Class 6       0.68      0.70      0.69       143
     Class 7       0.00      0.00      0.00         0
     Class 8       0.75      0.76      0.75        50
     Class 9       0.62      0.77      0.69       336
    Class 10       0.00      0.00      0.00         2
    Class 11       0.89      0.93      0.91        71
    Class 12       0.82      0.82      0.82       409
    Class 13       0.00      0.00      0.00         1
    Class 14       0.86      0.85      0.85       478
    Class 15       0.91      0.97      0.94       885
    Class 16       0.78      0.39      0.52        18
    